https://colab.research.google.com/drive/1VLG8e7YSEwypxU-noRNhsv5dW4NfTGce#scrollTo=8DKMc0fiej4N

# Настройка доступов и подгрузка библиотек

Введите название темплейта.

In [ ]:
meme_pattern = 'disaster girl'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ln -s "/content/drive/My Drive" /gdrive

In [ ]:
%tensorflow_version 1.x
!pip install git+https://github.com/SerSmith/gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

TensorFlow 1.x selected.
  Cloning https://github.com/SerSmith/gpt-2-simple to /tmp/pip-req-build-9ixmh8_s
  Running command git clone -q https://github.com/SerSmith/gpt-2-simple /tmp/pip-req-build-9ixmh8_s
  Created wheel for gpt-2-simple: filename=gpt_2_simple-0.7.1-cp36-none-any.whl size=25647 sha256=d0d04da76bace7a1e2df64015e96dc91f1c87f8e6bd6a7b548fa9fa63d832cb7
  Stored in directory: /tmp/pip-ephem-wheel-cache-cb7be4a0/wheels/05/e5/9d/9b6ebf15d32922867291a66ba17b8610a5621e6cd1a399fba5
Successfully built gpt-2-simple
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



# Подготовка датасета для дообучения

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import re
import os
import pickle
import nltk
import shutil
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

path_raw_data = '/gdrive/MADE_PROJECT/raw_data/memes_all_eng_v2.csv'
path_folder_gdisk = '/content/drive/My Drive'

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
df = pd.read_csv(path_raw_data)
df['template_name_prep'].value_counts()

batman slapping robin    18241
x x everywhere           16677
waiting skeleton         12314
change my mind           12169
disaster girl             9418
Name: template_name_prep, dtype: int64

In [ ]:
def prepare_text(text):
  text = str(text).strip().upper()
  text = re.sub(r"\s", " ", text)
  text = re.sub(r"\n", " ", text)
  text = re.sub(r"&", "and", text)
  text = re.sub(r"|", "", text)
  return text

def prepare_train(data, template, dest_path):
  out = df[df['template_name_prep'] == template]['text'].drop_duplicates().reset_index(drop=True).apply(prepare_text)
  print(out.shape)
  out.to_csv(dest_path, index=False, header=False)
  return out

q = prepare_train(df, meme_pattern, os.path.join(path_folder_gdisk, f'{meme_pattern}.csv'))
table = os.path.join(path_folder_gdisk, f'{meme_pattern}.csv')
!cat "$table" | head -n 10

(9372,)
SAID THEY HAD CORONA|I SOLVED THAT PROBLEM|
WHOLE WORLD|THE MAN WHO ATE A BAT|
YOU WANTED TEAMS INTEGRATION WITH JIRA?|YOU GOT IT|
EUROPEAN UNION|POPULISTS IN THE EU|
I BURNED DOWN YOUR HOUSE|SO YOU WOULD HAVE TO COME KEEP ME COMPANY|
HE WAS QUARANTINED IN HIS SIDE CHICKS HOUSE||
ME: SURELY WE HAVE AN INDEX ON COMMENTS.CREATED_AT DB001:||
HOW MY SIBLING TELL WHAT HAPPENED|WHEN WE PLAY FIGHT|
"LOST 10 GAMES IN A ROW|""LET'S PLAY ONE MORE""|"
CHARI|FELL OVER ND COMMITED ARSON|


Запуск дообучения

In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [ ]:
!nvidia-smi

Mon Nov 23 20:33:00 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
sess = gpt2.start_tf_sess()
gpt2.mount_gdrive()
gpt2.download_gpt2(model_name="124M")
gpt2.copy_file_from_gdrive(f'{meme_pattern}.csv')

history = gpt2.finetune(sess,
        f'{meme_pattern}.csv',
        steps=1500,
        model_name='124M',
        model_dir='models',
        combine=50000,
        batch_size=1,
        learning_rate=0.0001,
        accumulate_gradients=5,
        restore_from='latest',
        run_name=meme_pattern,
        checkpoint_dir='checkpoint',
        sample_every=50,
        sample_length=100,
        sample_num=500,
        multi_gpu=False,
        save_every=1000,
        print_every=50)

Fetching checkpoint: 1.05Mit [00:00, 463Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 140Mit/s]                                                    
Fetching hparams.json: 1.05Mit [00:00, 632Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001:   0%|                          | 0.00/498M [00:00<?, ?it/s]

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Fetching model.ckpt.data-00000-of-00001: 498Mit [00:07, 67.3Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 322Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 179Mit/s]                                                 
Fetching vocab.bpe: 1.05Mit [00:00, 207Mit/s]                                                       


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Loading checkpoint models/124M/model.ckpt
INFO:tensorflow:Restoring parameters from models/124M/model.ckpt


100%|██████████| 1/1 [00:00<00:00, 55.63it/s]

...

[1400 | 17226.38] loss=0.05 avg=0.50
ES|<|endoftext|>
<|startoftext|>WHEN MOMMY SAYS|YOU ARE DEMENTPHYXED AND HERD BLOOD ALCOHOL POQUETFOLLAS|<|endoftext|>
<|startoftext|>WORLD|COVID 19|<|endoftext|>
<|startoftext|>FOPPAS SERVER|LE CALLIS|<|endof


/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


[1450 | 17893.35] loss=0.05 avg=0.48
 HIS REAL IDENTITY<|endoftext|>
<|startoftext|>THERE WAS A CONVERSATION POTIED|IT WAS UNDERGROUND<|endoftext|>
<|startoftext|>THEY SAID WE NEEDED TO RE-THINK|MY MOM'S EXPERIENCE<|endoftext|>
<|startoftext|>"IF THIS HOUSE BELIEVES IN GHOST 3|HE SHOULD NEVER


/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


[1500 | 18561.68] loss=0.06 avg=0.46
Saving checkpoint/disaster girl/model-1500
Instructions for updating:
Use standard file APIs to delete files with this prefix.


In [ ]:
with open(f'/gdrive/{meme_pattern}_history.pickle', 'wb') as f:
  pickle.dump(history, f)

In [ ]:
gpt2.copy_checkpoint_to_gdrive(run_name=meme_pattern)